In [176]:
import pandas as pd
from pandas import Series, DataFrame
import datetime 
from datetime import datetime

In [178]:
import os
os.getcwd()

'/Users/chereowang27/NYDSA/Python_intro_project'

In [207]:
disney_p = pd.read_csv('disney_plus_shows.csv', header=0)
netflix = pd.read_csv('netflix_list.csv', header=0)

disney_p['added_at'] = disney_p["added_at"].apply(lambda s: datetime.strptime(s, "%d-%b-%y"))
disney_p['added_atyear'] = disney_p['added_at'].apply(lambda s: s.year)


hyphen = disney_p['year'].str.split(pat = '-', expand = True ).loc[30,0][4]
df_year = disney_p['year'].str.split(pat = hyphen, expand = True ) \
            .rename(columns = {0:'yearStart', 1: 'yearEnd'})
       

disney_p = pd.concat([disney_p,df_year], axis = 1)

#count the number of movies in each genre
d_movie = disney_p[disney_p['type']=='movie']
d_movie.head()

,imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,...,actors,language,country,awards,metascore,imdb_rating,imdb_votes,added_atyear,yearStart,yearEnd
0,tt0147800,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d...",movie,PG-13,1999,31-Mar-99,2019-11-12,97 min,"Comedy, Drama, Romance",...,"Heath Ledger, Julia Stiles, Joseph Gordon-Levi...","English, French",USA,2 wins & 13 nominations.,70.0,7.3,"283,945",2019,1999,None
2,tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal D...,movie,G,1996,27-Nov-96,2019-11-12,103 min,"Adventure, Comedy, Crime, Family",...,"Glenn Close, Jeff Daniels, Joely Richardson, J...","English, Spanish","USA, UK",Nominated for 1 Golden Globe. Another 3 wins &...,49.0,5.7,"97,785",2019,1996,None
3,tt0324941,101 Dalmatians 2: Patch's London Adventure,"Being one of 101 takes its toll on Patch, who ...",movie,G,2002,21-Jan-03,2019-11-12,74 min,"Animation, Adventure, Comedy, Family, Musical",...,"Barry Bostwick, Jason Alexander, Martin Short,...",English,USA,5 wins & 10 nominations.,NaN,5.8,"7,434",2019,2002,None
4,tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes afte...,movie,G,2000,22-Nov-00,2019-11-12,100 min,"Adventure, Comedy, Family",...,"Glenn Close, Gérard Depardieu, Ioan Gruffudd, ...",English,"USA, UK",Nominated for 1 Oscar. Another 1 win & 7 nomin...,35.0,4.9,"33,444",2019,2000,None
5,tt1846442,12 Dates of Christmas,"A story that follows Kate, a young woman who a...",movie,PG,2011,11-Dec-11,2019-11-12,90 min,"Comedy, Fantasy, Romance",...,"Laura Miyata, Vijay Mehta, Amy Smart, Audrey D...",English,USA,1 nomination.,NaN,6.3,"4,933",2019,2011,None


In [208]:
genres = set(d_movie['genre'].str.cat(sep = ', ').split(', '))
d_movie['genre'].str.contains('Comedy').mean()

0.5680473372781065

In [209]:
def g_count(col,gn):
    return col.str.contains(gn).sum()
  
#d_movie.groupby('added_atyear').agg(comedy = ('genre',lambda col: g_count(col,'Comedy')))



In [213]:
agg_df = []
for gn in genres:
    tmp = d_movie.groupby('added_atyear').agg(
        **{gn : ('genre', lambda col: g_count(col,gn))}
    )
    agg_df.append(tmp)


In [215]:
pd.concat(agg_df, axis=1)


,Biography,Documentary,History,Action,Mystery,Romance,Musical,Animation,Music,Horror,...,Sport,Family,Western,Adventure,War,Fantasy,Sci-Fi,Drama,Short,Reality-TV
added_atyear,,,,,,,,,,,,,,,,,,,,,
2019,16,49,8,74,19,77,104,226,125,7,...,43,470,13,238,3,185,88,134,86,2
2020,2,16,6,15,3,10,4,34,6,1,...,3,60,1,33,1,16,8,12,27,0
